In [1]:
%pip install -q pandas
%pip install -q matplotlib
%pip install -q seaborn
%pip install -q scikit-learn
%pip install -q keras
%pip install -q xgboost
%pip install -q catboost
%pip install -q lightgbm
%pip install -q wordcloud
%pip install -q nltk
%pip install -q imblearn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import RandomOverSampler

# Read the data into a dataframe
df = pd.read_csv("cleaned_data.csv")

# Separate the features and target variable
X = df[
    [
        "total_direct_mentions",
        "total_indirect_mentions",
        "total_likes",
        "total_retweets",
        "total_project_followers",
        "total_indirect_followers",
        "total_positive_direct_mentions",
        "total_negative_direct_mentions",
        "total_positive_indirect_mentions",
        "total_negative_indirect_mentions",
        "soft_cap",
    ]
]
y = df["ico_success"]

# Perform Random Oversampling
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

# Define the parameter grids for grid search
rf_param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [5, 10],
    "min_samples_split": [2, 5, 10],
}

lr_param_grid = {"C": [0.1, 1, 10], "penalty": ["l1", "l2"]}

nb_param_grid = {}

svm_param_grid = {
    "C": [0.1, 1, 10],
    "gamma": [1e-2, 1e-3, 1e-4, 1e-5],
    "kernel": ["rbf"],
}

# Define the models
models = {
    "Random Forest": (RandomForestClassifier(random_state=42), rf_param_grid),
    "Logistic Regression": (LogisticRegression(random_state=42), lr_param_grid),
    "Naïve Bayes": (GaussianNB(), nb_param_grid),
    "SVM": (SVC(random_state=42), svm_param_grid),
}

# Perform grid search and cross-validation for each model
for model_name, (model, param_grid) in models.items():
    print(f"Model: {model_name}")
    print("=" * 50)

    # Perform grid search with 5-fold cross-validation
    grid_search = GridSearchCV(
        estimator=model, param_grid=param_grid, cv=5, scoring="accuracy"
    )
    grid_search.fit(X_resampled, y_resampled)

    # Print the best parameters and the corresponding score
    print("Best Parameters: ", grid_search.best_params_)
    print()

    # Perform 5-fold cross-validation with the best model
    cv_results = cross_validate(
        grid_search.best_estimator_,
        X_resampled,
        y_resampled,
        cv=5,
        scoring=["accuracy", "precision", "recall", "f1"],
    )
    print("Cross-Validation Results:")
    print("=" * 50)
    overfitted_folds = 0  # Counter for overfitted folds
    for fold_idx, fold_result in enumerate(cv_results["test_accuracy"]):
        if fold_result == 1.0:  # Check for overfitted fold
            overfitted_folds += 1
            continue  # Skip overfitted fold

        print(f"Fold {fold_idx+1}:")
        print(f"Accuracy: {fold_result}")
        print(f"Precision: {cv_results['test_precision'][fold_idx]}")
        print(f"Recall: {cv_results['test_recall'][fold_idx]}")
        print(f"F1 Score: {cv_results['test_f1'][fold_idx]}")
        print()

    # Calculate average results across non-overfitted folds
    num_folds = len(cv_results["test_accuracy"])
    num_non_overfitted_folds = num_folds - overfitted_folds
    avg_accuracy = (
        sum(cv_results["test_accuracy"]) - (overfitted_folds * 1)
    ) / num_non_overfitted_folds
    avg_precision = (
        sum(cv_results["test_precision"]) - (overfitted_folds * 1)
    ) / num_non_overfitted_folds
    avg_recall = (
        sum(cv_results["test_recall"]) - (overfitted_folds * 1)
    ) / num_non_overfitted_folds
    avg_f1 = (
        sum(cv_results["test_f1"]) - (overfitted_folds * 1)
    ) / num_non_overfitted_folds

    # Print the average results
    print(f"Average Accuracy: {avg_accuracy}")
    print(f"Average Precision: {avg_precision}")
    print(f"Average Recall: {avg_recall}")
    print(f"Average F1 Score: {avg_f1}")
    print()

Model: Random Forest
Best Parameters:  {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 100}

Cross-Validation Results:
Fold 1:
Accuracy: 0.8927038626609443
Precision: 0.8709677419354839
Recall: 0.9230769230769231
F1 Score: 0.8962655601659751

Fold 2:
Accuracy: 0.9012875536480687
Precision: 0.8983050847457628
Recall: 0.905982905982906
F1 Score: 0.902127659574468

Fold 3:
Accuracy: 0.9613733905579399
Precision: 0.9908256880733946
Recall: 0.9310344827586207
F1 Score: 0.96

Fold 4:
Accuracy: 0.8369098712446352
Precision: 0.9875
Recall: 0.6810344827586207
F1 Score: 0.8061224489795918

Fold 5:
Accuracy: 0.8706896551724138
Precision: 1.0
Recall: 0.7413793103448276
F1 Score: 0.8514851485148515

Average Accuracy: 0.8925928666568004
Average Precision: 0.9495197029509281
Average Recall: 0.8365016209843796
Average F1 Score: 0.8832001634469773

Model: Logistic Regression


/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/l

Best Parameters:  {'C': 10, 'penalty': 'l2'}

Cross-Validation Results:
Fold 1:
Accuracy: 0.7296137339055794
Precision: 0.7547169811320755
Recall: 0.6837606837606838
F1 Score: 0.7174887892376682

Fold 2:
Accuracy: 0.7424892703862661
Precision: 0.7663551401869159
Recall: 0.7008547008547008
F1 Score: 0.7321428571428571

Fold 3:
Accuracy: 0.6995708154506438
Precision: 0.75
Recall: 0.5948275862068966
F1 Score: 0.6634615384615384

Fold 4:
Accuracy: 0.5836909871244635
Precision: 0.6862745098039216
Recall: 0.3017241379310345
F1 Score: 0.41916167664670656

Fold 5:
Accuracy: 0.6594827586206896
Precision: 0.7936507936507936
Recall: 0.43103448275862066
F1 Score: 0.5586592178770949

Average Accuracy: 0.6829695130975285
Average Precision: 0.7501994849547413
Average Recall: 0.5424403183023873
Average F1 Score: 0.618182815873173

Model: Naïve Bayes
Best Parameters:  {}

Cross-Validation Results:
Fold 1:
Accuracy: 0.5278969957081545
Precision: 0.5660377358490566
Recall: 0.2564102564102564
F1 Score: 0.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import plotly.express as px
import plotly.graph_objects as go
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import sklearn.metrics as mt
import nltk
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.svm import LinearSVC
from collections import Counter
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import AdaBoostClassifier

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=42
)

In [5]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn import metrics
import time


def train_and_evaluate_model(model, x_train, x_test, y_train, y_test):
    # Train the model on the training data
    now = time.time()
    model.fit(x_train, y_train)
    print("Training time: ", time.time() - now)

    # Evaluate the trained model on the test data
    y_pred = model.predict(x_test)

    # test recall and f1-score
    test_acc = metrics.accuracy_score(y_test, y_pred)
    test_prec = metrics.precision_score(y_test, y_pred)
    test_recall = metrics.recall_score(y_test, y_pred)
    test_f1 = metrics.f1_score(y_test, y_pred, average="weighted")

    # train recall and f1-score
    y_pred_train = model.predict(x_train)
    train_acc = metrics.accuracy_score(y_train, y_pred_train)
    train_prec = metrics.precision_score(y_train, y_pred_train)
    train_recall = metrics.recall_score(y_train, y_pred_train)
    train_f1 = metrics.f1_score(y_train, y_pred_train, average="weighted")

    # print the metrics
    print("Test accuracy: ", test_acc)
    print("Test precision: ", test_prec)
    print("Test recall: ", test_recall)
    print("Test f1-score: ", test_f1)
    print("Train accuracy: ", train_acc)
    print("Train precision: ", train_prec)
    print("Train recall: ", train_recall)
    print("Train f1-score: ", train_f1)

    # Return the trained model
    return model

In [6]:
from sklearn.ensemble import StackingClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

models = {
    "AdaBoost": AdaBoostClassifier(
        algorithm="SAMME",
        base_estimator=RandomForestClassifier(random_state=123),
        learning_rate=1,
        n_estimators=10,
        random_state=123,
    ),
    "Adaboost_GNB": AdaBoostClassifier(
        base_estimator=GaussianNB(var_smoothing=0.012328467394420659), random_state=42
    ),
    "XGBboost": XGBClassifier(
        random_state=42, learning_rate=0.03, max_depth=3, n_estimators=300, reg_lambda=2
    ),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "CatBoost": CatBoostClassifier(
        random_state=42,
        iterations=200,
        depth=4,
        loss_function="Logloss",
        l2_leaf_reg=1e-20,
        leaf_estimation_iterations=10,
        logging_level="Silent",
        learning_rate=0.03,
    ),
    "Random Forest": RandomForestClassifier(random_state=42),
    "StackingClassifier": StackingClassifier(
        estimators=[
            ("Decision Tree", DecisionTreeClassifier(random_state=42)),
            ("K-Nearest Neighbors", KNeighborsClassifier()),
        ]
    ),
    "LightGBM": LGBMClassifier(random_state=42),
}
for model in models:
    print(model)
    models[model] = train_and_evaluate_model(
        models[model], X_train, X_test, y_train, y_test
    )

AdaBoost
Training time:  0.1605207920074463
Test accuracy:  0.7647058823529411
Test precision:  0.7844311377245509
Test recall:  0.916083916083916
Test f1-score:  0.7450025171358865
Train accuracy:  1.0
Train precision:  1.0
Train recall:  1.0
Train f1-score:  1.0
Adaboost_GNB


/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Training time:  0.05511021614074707
Test accuracy:  0.29411764705882354
Test precision:  0.0
Test recall:  0.0
Test f1-score:  0.1359180035650624
Train accuracy:  0.2818627450980392
Train precision:  1.0
Train recall:  0.010135135135135136
Train f1-score:  0.13349488116860375
XGBboost
Training time:  0.8093397617340088
Test accuracy:  0.75
Test precision:  0.7674418604651163
Test recall:  0.9230769230769231
Test f1-score:  0.7225294418842807
Train accuracy:  0.9191176470588235
Train precision:  0.9096573208722741
Train recall:  0.9864864864864865
Train f1-score:  0.9156757186638694
Gradient Boosting
Training time:  0.23137974739074707
Test accuracy:  0.7573529411764706
Test precision:  0.7791044776119403
Test recall:  0.9125874125874126
Test f1-score:  0.7364395479484993
Train accuracy:  0.9877450980392157
Train precision:  0.9833887043189369
Train recall:  1.0
Train f1-score:  0.9876565205830387
CatBoost
Training time:  0.2796640396118164
Test accuracy:  0.7941176470588235
Test precis

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import (
    AdaBoostClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier,
)
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from lightgbm import LGBMClassifier

# Define the models with their initial parameter settings
models = {
    "AdaBoost": AdaBoostClassifier(
        algorithm="SAMME",
        base_estimator=RandomForestClassifier(random_state=123),
        learning_rate=1,
        n_estimators=10,
        random_state=123,
    ),
    "Adaboost_GNB": AdaBoostClassifier(
        base_estimator=GaussianNB(var_smoothing=0.012328467394420659), random_state=42
    ),
    "XGBboost": XGBClassifier(
        random_state=42, learning_rate=0.03, max_depth=3, n_estimators=300, reg_lambda=2
    ),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "CatBoost": CatBoostClassifier(
        random_state=42,
        iterations=200,
        depth=4,
        loss_function="Logloss",
        l2_leaf_reg=1e-20,
        leaf_estimation_iterations=10,
        logging_level="Silent",
        learning_rate=0.03,
    ),
    "Random Forest": RandomForestClassifier(random_state=42),
    "StackingClassifier": StackingClassifier(
        estimators=[
            ("Decision Tree", DecisionTreeClassifier(random_state=42)),
            ("K-Nearest Neighbors", KNeighborsClassifier()),
        ]
    ),
    "LightGBM": LGBMClassifier(random_state=42),
}

# Define the parameter grids for grid search
param_grids = {
    "AdaBoost": {"n_estimators": [10, 50, 100], "learning_rate": [0.1, 0.5, 1.0]},
    "Adaboost_GNB": {},
    "XGBboost": {
        "n_estimators": [100, 200, 300],
        "learning_rate": [0.01, 0.03, 0.1],
        "max_depth": [3, 5, 10],
        "reg_lambda": [1, 2, 5],
    },
    "Gradient Boosting": {},
    "CatBoost": {
        "iterations": [100, 200, 300],
        "learning_rate": [0.01, 0.03, 0.1],
        "depth": [3, 5, 10],
    },
    "Random Forest": {},
    "StackingClassifier": {},
    "LightGBM": {},
}

# Perform grid search for each model
for model_name, model in models.items():
    print(f"Model: {model_name}")
    print("=" * 50)

    # Get the parameter grid for the current model
    param_grid = param_grids[model_name]

    # Perform grid search with 5-fold cross-validation
    grid_search = GridSearchCV(
        estimator=model, param_grid=param_grid, cv=5, scoring="accuracy"
    )
    grid_search.fit(X_resampled, y_resampled)

    # Print the best parameters and the corresponding score
    print("Best Parameters: ", grid_search.best_params_)
    print()

    # Perform 5-fold cross-validation with the best model
    cv_results = cross_validate(
        grid_search.best_estimator_,
        X_resampled,
        y_resampled,
        cv=5,
        scoring=["accuracy", "precision", "recall", "f1"],
    )
    print("Cross-Validation Results:")
    print("=" * 50)
    overfitted_folds = 0  # Counter for overfitted folds
    for fold_idx, fold_result in enumerate(cv_results["test_accuracy"]):
        if fold_result == 1.0:  # Check for overfitted fold
            overfitted_folds += 1
            continue  # Skip overfitted fold

        print(f"Fold {fold_idx+1}:")
        print(f"Accuracy: {fold_result}")
        print(f"Precision: {cv_results['test_precision'][fold_idx]}")
        print(f"Recall: {cv_results['test_recall'][fold_idx]}")
        print(f"F1 Score: {cv_results['test_f1'][fold_idx]}")
        print()

    # Calculate average results across non-overfitted folds
    num_folds = len(cv_results["test_accuracy"])
    num_non_overfitted_folds = num_folds - overfitted_folds
    avg_accuracy = (
        sum(cv_results["test_accuracy"]) - (overfitted_folds * 1)
    ) / num_non_overfitted_folds
    avg_precision = (
        sum(cv_results["test_precision"]) - (overfitted_folds * 1)
    ) / num_non_overfitted_folds
    avg_recall = (
        sum(cv_results["test_recall"]) - (overfitted_folds * 1)
    ) / num_non_overfitted_folds
    avg_f1 = (
        sum(cv_results["test_f1"]) - (overfitted_folds * 1)
    ) / num_non_overfitted_folds

    # Print the average results
    print(f"Average Accuracy: {avg_accuracy}")
    print(f"Average Precision: {avg_precision}")
    print(f"Average Recall: {avg_recall}")
    print(f"Average F1 Score: {avg_f1}")
    print()

Model: AdaBoost


/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was ren

Best Parameters:  {'learning_rate': 0.1, 'n_estimators': 10}



/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was ren

Cross-Validation Results:
Fold 1:
Accuracy: 0.8927038626609443
Precision: 0.8709677419354839
Recall: 0.9230769230769231
F1 Score: 0.8962655601659751

Fold 2:
Accuracy: 0.9012875536480687
Precision: 0.9051724137931034
Recall: 0.8974358974358975
F1 Score: 0.9012875536480686

Fold 3:
Accuracy: 0.9656652360515021
Precision: 1.0
Recall: 0.9310344827586207
F1 Score: 0.9642857142857143

Fold 4:
Accuracy: 0.8283261802575107
Precision: 1.0
Recall: 0.6551724137931034
F1 Score: 0.7916666666666666

Fold 5:
Accuracy: 0.875
Precision: 1.0
Recall: 0.75
F1 Score: 0.8571428571428571

Average Accuracy: 0.8925965665236053
Average Precision: 0.9552280311457174
Average Recall: 0.8313439434129088
Average F1 Score: 0.8821296703818563

Model: Adaboost_GNB


/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was ren

Best Parameters:  {}



/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/inflaton/miniconda3/envs/cs701/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was ren

Cross-Validation Results:
Fold 1:
Accuracy: 0.51931330472103
Precision: 0.5109170305676856
Recall: 1.0
F1 Score: 0.676300578034682

Fold 2:
Accuracy: 0.4892703862660944
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

Fold 3:
Accuracy: 0.463519313304721
Precision: 0.4117647058823529
Recall: 0.1810344827586207
F1 Score: 0.25149700598802394

Fold 4:
Accuracy: 0.5150214592274678
Precision: 0.5384615384615384
Recall: 0.1810344827586207
F1 Score: 0.2709677419354839

Fold 5:
Accuracy: 0.4870689655172414
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

Average Accuracy: 0.49483868580731094
Average Precision: 0.2922286549823154
Average Recall: 0.2724137931034483
Average F1 Score: 0.23975306519163797

Model: XGBboost
Best Parameters:  {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'reg_lambda': 5}

Cross-Validation Results:
Fold 1:
Accuracy: 0.8841201716738197
Precision: 0.8629032258064516
Recall: 0.9145299145299145
F1 Score: 0.8879668049792532

Fold 2:
Accuracy: 0.8969957081545065
Precision: 0.

In [8]:
models = {
    "XGBboost": XGBClassifier(
        random_state=42, learning_rate=0.03, max_depth=3, n_estimators=300, reg_lambda=2
    ),
    "CatBoost": CatBoostClassifier(
        random_state=42,
        iterations=200,
        depth=4,
        loss_function="Logloss",
        l2_leaf_reg=1e-20,
        leaf_estimation_iterations=10,
        logging_level="Silent",
        learning_rate=0.03,
    ),
    "Random Forest": RandomForestClassifier(random_state=42),
}

# Define the parameter grids for grid search
param_grids = {
    "XGBboost": {
        "n_estimators": [100, 200, 300],
        "learning_rate": [0.01, 0.03, 0.1],
        "max_depth": [3, 5, 10],
        "reg_lambda": [1, 2, 5],
    },
    "CatBoost": {
        "iterations": [100, 200, 300],
        "learning_rate": [0.01, 0.03, 0.1],
        "depth": [3, 5, 10],
    },
    "Random Forest": {},
}

# Perform grid search for each model
for model_name, model in models.items():
    print(f"Model: {model_name}")
    print("=" * 50)

    # Get the parameter grid for the current model
    param_grid = param_grids[model_name]

    # Perform grid search with 5-fold cross-validation
    grid_search = GridSearchCV(
        estimator=model, param_grid=param_grid, cv=5, scoring="accuracy"
    )
    grid_search.fit(X_resampled, y_resampled)

    # Print the best parameters and the corresponding score
    print("Best Parameters: ", grid_search.best_params_)
    print()

    # Perform 5-fold cross-validation with the best model
    cv_results = cross_validate(
        grid_search.best_estimator_,
        X_resampled,
        y_resampled,
        cv=5,
        scoring=["accuracy", "precision", "recall", "f1"],
    )
    print("Cross-Validation Results:")
    print("=" * 50)
    overfitted_folds = 0  # Counter for overfitted folds
    for fold_idx, fold_result in enumerate(cv_results["test_accuracy"]):
        if fold_result == 1.0:  # Check for overfitted fold
            overfitted_folds += 1
            continue  # Skip overfitted fold

        print(f"Fold {fold_idx+1}:")
        print(f"Accuracy: {fold_result}")
        print(f"Precision: {cv_results['test_precision'][fold_idx]}")
        print(f"Recall: {cv_results['test_recall'][fold_idx]}")
        print(f"F1 Score: {cv_results['test_f1'][fold_idx]}")
        print()

    # Calculate average results across non-overfitted folds
    num_folds = len(cv_results["test_accuracy"])
    num_non_overfitted_folds = num_folds - overfitted_folds
    avg_accuracy = (
        sum(cv_results["test_accuracy"]) - (overfitted_folds * 1)
    ) / num_non_overfitted_folds
    avg_precision = (
        sum(cv_results["test_precision"]) - (overfitted_folds * 1)
    ) / num_non_overfitted_folds
    avg_recall = (
        sum(cv_results["test_recall"]) - (overfitted_folds * 1)
    ) / num_non_overfitted_folds
    avg_f1 = (
        sum(cv_results["test_f1"]) - (overfitted_folds * 1)
    ) / num_non_overfitted_folds

    # Print the average results
    print(f"Average Accuracy: {avg_accuracy}")
    print(f"Average Precision: {avg_precision}")
    print(f"Average Recall: {avg_recall}")
    print(f"Average F1 Score: {avg_f1}")
    print()

Model: XGBboost
Best Parameters:  {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'reg_lambda': 5}

Cross-Validation Results:
Fold 1:
Accuracy: 0.8841201716738197
Precision: 0.8629032258064516
Recall: 0.9145299145299145
F1 Score: 0.8879668049792532

Fold 2:
Accuracy: 0.8969957081545065
Precision: 0.911504424778761
Recall: 0.8803418803418803
F1 Score: 0.8956521739130435

Fold 3:
Accuracy: 0.9356223175965666
Precision: 1.0
Recall: 0.8706896551724138
F1 Score: 0.9308755760368664

Fold 4:
Accuracy: 0.8283261802575107
Precision: 1.0
Recall: 0.6551724137931034
F1 Score: 0.7916666666666666

Fold 5:
Accuracy: 0.896551724137931
Precision: 1.0
Recall: 0.7931034482758621
F1 Score: 0.8846153846153846

Average Accuracy: 0.888323220364067
Average Precision: 0.9548815301170425
Average Recall: 0.8227674624226348
Average F1 Score: 0.8781553212422428

Model: CatBoost
Best Parameters:  {'depth': 10, 'iterations': 300, 'learning_rate': 0.01}

Cross-Validation Results:
Fold 1:
Accuracy: 0.88412